# Shareability of demand patterns
---

In [1]:
import sys
sys.path.append('/Users/Jaime/Google Drive/GitHub/Shareability')
import pandas as pd
pd.options.mode.chained_assignment = None
pd.__version__
#Should be 0.25 (deprecated functions)

'1.0.5'

In [5]:
from ExMAS.utils import get_config

In [6]:
params = get_config('../configs/experiments.json')
params = make_paths(params)
import pprint
pprint.pprint(params.shareability)

FileNotFoundError: [Errno 2] No such file or directory: '../configs/experiments.json'

In [16]:
inData = load_G(inData, params, stats = True) # download the Amsterdam graph

----
### Params

In [17]:
params.t0 = pd.Timestamp.now()
params.nP = 1000 # number of trips
params.simTime = 1 #hours
params.shareability.avg_speed = 8
params.shareability.shared_discount = 0.3
params.shareability.delay_value = 1.5
params.shareability.WtS = 1.2
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 1200


In [18]:
params.shareability.max_degree = 8
params.shareability.nP = 200

In [19]:
params.t0 = pd.to_datetime('17:00')
params.simTime = 1
#inData = load_albatross_csv(inData,params, sample = True)
#inData = generate_demand(inData, params)


In [30]:
origins_albatross= pd.read_csv('AM_origins.csv', index_col = 0)
origins_albatross = origins_albatross[origins_albatross['origin'].isin(inData.skim.index)] #droping those origins not in skim
len(origins_albatross)
#Sample_origins = origins_albatross.sample(params.nP)
#Sample_origins.to_csv('Sample_origins.csv')
Sample_origins= pd.read_csv('Sample_origins.csv', index_col = 0)


In [33]:
Centers = pd.DataFrame(data={'name':['Dam Square','Station Zuid', 'Concertgebouw', 'Sloterdijk'],'x':[4.8909126,4.871887,4.8768717,4.8351158], 'y':[52.373095,52.338948,52.3563125,52.3888349]})
Centers['node'] = Centers.apply(lambda center: get_nearest_node(inData.G, (center.y, center.x)), axis=1)
Centers

,name,x,y,node
0,Dam Square,4.890913,52.373095,3176714997
1,Station Zuid,4.871887,52.338948,295391250
2,Concertgebouw,4.876872,52.356313,46320321
3,Sloterdijk,4.835116,52.388835,46462406


In [34]:
def synthetic_demand_poly_IV(_inData, _params=None):
   
   # 1. create a passenger DataFrame
    
    df = pd.DataFrame(index=np.arange(1, _params.nP + 1), columns=_inData.passengers.columns)
    df.status = 0
    
    # initialize with a completely random nodes (no distribution)
    df.pos = df.apply(lambda x: rand_node(_inData.nodes), axis=1)
    _inData.passengers = df

	
    print('1',pd.Timestamp.now())

    # 1. create a Centers and potential destinations
    Centers = pd.DataFrame(data={'name':['Dam Square','Station Zuid', 'Concertgebouw', 'Sloterdijk'],'x':[4.8909126,4.871887,4.8790061,4.8351158], 'y':[52.373095,52.338948,52.356275,52.3888349]})
    Centers['node'] = Centers.apply(lambda center: get_nearest_node(inData.G, (center.y, center.x)), axis=1)
    
    #drop not considered centers
    Centers = Centers[Centers.index.isin(range(_params.nCenters))]

    Dist = []
    
    rdist = gamma_random(a=_params.gammdist.shape, scale = _params.gammdist.scale)
    
    for i in range(_params.nCenters):
              
        # 2. distances and probabiltiies for each node #get distances from centre for each node
        _distances = _inData.skim.transpose()[Centers['node'][i]].to_frame().dropna()  # compute distances from center
        _distances.columns = ['distance']
        _distances = _distances[_distances['distance'] < _params.dist_threshold]  # drop disconnected points from choice set
    
        # apply distributions to obtain probability of destination
        _distances['p_destination'] = _distances['distance'].apply(lambda x:rdist.pdf(x))
        
        _distances['dist_range'] = _distances['distance'].apply(lambda x: math.trunc(x/500) + 1 )
        _distances_sum = _distances.groupby(by=['dist_range']).count()
        _distances_sum = _distances_sum.reset_index()
        _distances_sum = _distances_sum.drop(columns='p_destination')
        _distances_sum.columns = ['dist_range', 'cant']
        total = _distances_sum['cant'].mean()
        index_id = _distances.index
        _distances = _distances.merge(_distances_sum, on='dist_range', how='left')
        _distances.index = index_id
        _distances['fact'] = total/_distances['cant']
        _distances['p_destination_fixed'] = _distances['p_destination'] * _distances['fact']

        Dist.append(_distances)
    

    print('2',pd.Timestamp.now())

    randomdestinations = []
   
    #we can generate more potential destinations if desired
    for i in range(_params.nCenters):
        for j in range(_params.nP*3):
            randomdestinations.append(Dist[i].sample(1, weights='p_destination_fixed', replace=True).index.values[0])     

    print('3',pd.Timestamp.now())

    _inData.RandomDestinations = randomdestinations

    # 3. create requests DataFrame
    requests = pd.DataFrame(index=df.index, columns=_inData.requests.columns)
    
    # 3.a. draw random departure time
    
    # apply uniform distribution on request times
    treq = np.random.uniform(-_params.simTime * 60 * 60 / 2, _params.simTime * 60 * 60 / 2, _params.nP)  
    
    requests.treq = [_params.t0 + pd.Timedelta(int(_), 's') for _ in treq]

    # 3.b. draw origin sampling albatross origins
    requests.origin = Sample_origins.values

    # 3.c. draw destination with a probability
    #seed fixed or not?
    seed(1)
    
    destination = []
    dist = []

    print('4',pd.Timestamp.now())

    rdist = gamma_random(a=_params.gamma_imp.shape, scale = _params.gamma_imp.scale)

    for i in range(_params.nP):
    
        _distances = pd.DataFrame(randomdestinations)
        _distances.columns = ["id"]
        _distances['distance'] = _distances.apply(lambda request: _inData.skim.loc[requests.origin[i+1], request.id], axis=1)
        _distances = _distances[_distances['distance'] < _params.dist_threshold]
        
        while (sum(_distances['distance']) == 0):
            requests.origin[i+1] = origins_albatross['origin'].sample(1, replace=True).values[0]
            _distances = pd.DataFrame(randomdestinations)
            _distances.columns = ["id"]
            _distances['distance'] = _distances.apply(lambda request: _inData.skim.loc[requests.origin[i+1], request.id], axis=1)
            _distances = _distances[_distances['distance'] < _params.dist_threshold]
            #print('loop gravity',requests.origin[i+1],sum(_distances['distance']) ,pd.Timestamp.now())

        # apply distributions to obtain probability of destination
        
        _distances['p_destination'] = _distances['distance'].apply(lambda x:rdist.pdf(x))

        _distances['dist_range'] = _distances['distance'].apply(lambda x: math.trunc(x/500) + 1 )
        _distances_sum = _distances.groupby(by=['dist_range']).count()
        _distances_sum = _distances_sum.reset_index()
        _distances_sum = _distances_sum.drop(columns=['id','p_destination'])
        _distances_sum.columns = ['dist_range', 'cant']
        _distances_sum

        total = _distances_sum['cant'].mean()
        index_id = _distances.index
        _distances = _distances.merge(_distances_sum, on='dist_range', how='left')
        _distances.index = index_id
        _distances['fact'] = total/_distances['cant']
        _distances['p_destination_fixed'] = _distances['p_destination'] * _distances['fact']
            
        samp_destination = _distances.sample(1, weights='p_destination_fixed', replace=True)
        destination.append(samp_destination['id'].values[0])
        dist.append(samp_destination['distance'].values[0])
        #print('iter ',i,pd.Timestamp.now())


    print('5',pd.Timestamp.now())


    requests.destination = destination
    requests.dist = dist
    requests['dist'] = dist
    #requests['dist'] = requests.apply(lambda request: _inData.skim.loc[request.origin, request.destination], axis=1)


    # 4 make travel times (without using speed, speed param is used inside cal_sblt)
    requests['ttrav'] = requests.apply(lambda request: pd.Timedelta(request.dist, 's').floor('s'), axis=1)

    requests.tarr = [request.treq + request.ttrav for _, request in requests.iterrows()]
    
    requests = requests.sort_values('treq')
    requests.index = df.index
      
    # output
    _inData.requests = requests
    _inData.passengers.pos = requests.origin
    _inData.probs = requests.dist
    _inData.centers = Centers
    
    print('6',pd.Timestamp.now())

    return _inData

In [35]:
nCenters_values = [1,2,3,4]

params.gammdist.scale = 1300
gammdist_shape_values = [1.5, 2.5 ,4 , 6]

params.gamma_imp.scale = 3200 
gamma_imp_shape_values = [1, 1.15 ,2 , 3]

inData_initial = inData

In [43]:
experiment = 1

for i in range(len(nCenters_values)):
    for j in range(len(gammdist_shape_values)):
        for k in range(len(gamma_imp_shape_values)):    
            print(f'Experiment {experiment}')
            experiment = experiment + 1

            params.nCenters = nCenters_values[i]
            params.gammdist.shape = gammdist_shape_values[j]
            params.gamma_imp.shape = gamma_imp_shape_values[k]

            inData = synthetic_demand_poly_IV(inData_initial, params)
            inData.requests.to_csv(f'requests_nCenters_{nCenters_values[i]}_gammdist_shape_{gammdist_shape_values[j]}_gamma_imp_shape_{gamma_imp_shape_values[k]}.csv')
            print(f'requests_nCenters_{nCenters_values[i]}_gammdist_shape_{gammdist_shape_values[j]}_gamma_imp_shape_{gamma_imp_shape_values[k]}.csv')        

Experiment 1
1 2020-10-06 16:26:02.099229
2 2020-10-06 16:26:03.925081
3 2020-10-06 16:26:09.353205
4 2020-10-06 16:26:09.374226
5 2020-10-06 16:35:44.371533
6 2020-10-06 16:35:44.600996
requests_nCenters_1_gammdist_shape_1.5_gamma_imp_shape_1.csv
Experiment 2
1 2020-10-06 16:35:44.824309
2 2020-10-06 16:35:46.647588
3 2020-10-06 16:35:52.449489
4 2020-10-06 16:35:52.470472
5 2020-10-06 16:45:35.691544
6 2020-10-06 16:45:35.874763
requests_nCenters_1_gammdist_shape_1.5_gamma_imp_shape_1.15.csv
Experiment 3
1 2020-10-06 16:45:36.115610
2 2020-10-06 16:45:38.043538
3 2020-10-06 16:45:44.344851
4 2020-10-06 16:45:44.368258
5 2020-10-06 16:54:33.662966
6 2020-10-06 16:54:33.829286
requests_nCenters_1_gammdist_shape_1.5_gamma_imp_shape_2.csv
Experiment 4
1 2020-10-06 16:54:34.007740
2 2020-10-06 16:54:35.714562
3 2020-10-06 16:54:40.168897
4 2020-10-06 16:54:40.189721
5 2020-10-06 17:03:25.157985
6 2020-10-06 17:03:25.344461
requests_nCenters_1_gammdist_shape_1.5_gamma_imp_shape_3.csv
Exper